In [ ]:
!pip install pydub

In [ ]:
!pip install transformers

In [39]:
! pip install gradio -q


In [40]:
import soundfile as sf
import librosa
from pydub import AudioSegment
import torch

from IPython.display import Audio
from scipy.io import wavfile
import numpy as np

import gradio as gr
import time

from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
#load pre-trained model and tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [42]:
def transcribe(audio, length):

  Audio(audio)

  data = wavfile.read(audio)
  framerate = data[0]
  sounddata = data[1]
  time = np.arange(0,len(sounddata))/framerate
  print(framerate)
  total_time = len(sounddata)/framerate
  print('Input length:', length)
  print('Total time:', len(sounddata)/framerate)

  if (total_time > length):
      return "Input audio is longer than suggested length"

  input_audio,_ = librosa.load(audio, sr=16000)

  input_values = tokenizer(input_audio, return_tensors="pt").input_values
  logits = model(input_values).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  text = tokenizer.batch_decode(predicted_ids)[0]

  print(text)

  return text

In [ ]:
gr.Interface(
    title = 'Real-time AI-based Audio Transcription, Recognition and Translation Web App',
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="microphone", type="filepath"),
        gr.Number(value=60)
    ],
    outputs=[
        "textbox"
    ],
    live=True).launch()